In [1]:
%load_ext dotenv
%dotenv

## Import Dependencies

In [2]:
import os
import re
import pprint

# Imports for IBM Watson's Speech to Text Transcriber
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

import text2emotion as te

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Transcription of the audio clips and detection of emotions from the transcripts

## Setup STT Service

In [3]:
api_key = os.getenv("api_key")
url = os.getenv("url")

In [4]:
# Setup Service
authenticator = IAMAuthenticator(api_key)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

## Open Audio Clips directory and sort file names

In [5]:
# Sorts the file names numerically
def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]

In [6]:
# Enter the absolute path to the temp folder containing the clips
folder_path = r'..\..\assets\temp'

file_names = os.listdir(folder_path)

file_names.sort(key=natural_key)

results = {}

## Transcription and Detection of Emotions

In [7]:
# Don't run this loop with alll the clip files in the temp folder
count = 0
for file_name in file_names:
    if file_name[-3:] == 'wav' and count < 2:

        file = os.path.abspath(os.path.join(folder_path, file_name))

        # Perform transcription
        with open(file, 'rb') as f:
            res = stt.recognize(audio=f, content_type='audio/wav', model='en-US_NarrowbandModel', continuous=True).get_result()

        text = ""
        conf = 0.0
    
        for i in range(len(res['results'])):
            text += res['results'][i]['alternatives'][0]['transcript'][0:-1] + ". "
            conf += res['results'][i]['alternatives'][0]['confidence']
            
        conf = conf/len(res['results'])

        emotions = te.get_emotion(text)
        
        final_emotion = max(emotions, key= lambda x: emotions[x])
        
        results[file_name] = {'Transcription': text, 'Confidence': conf, 'Emotions': emotions, 'Final_Emotion': final_emotion}
        
        count += 1

In [8]:
pprint.pprint(results)

{'clip_1.wav': {'Confidence': 0.9299999999999999,
                'Emotions': {'Angry': 0.0,
                             'Fear': 0.2,
                             'Happy': 0.0,
                             'Sad': 0.4,
                             'Surprise': 0.4},
                'Final_Emotion': 'Surprise',
                'Transcription': 'Alice in Wonderland by Lewis Carroll chapter '
                                 'one down the rabbit hole. Alice was '
                                 'beginning to get very tired of sitting by '
                                 'her sister on the bank and of having nothing '
                                 'to do once or twice she had peeped into the '
                                 'book her sister was reading but it had no '
                                 'pictures or conversations in it and what is '
                                 'the use of a book thought Alice without '
                                 'pictures our conversation so 